![Cartoon of telecom customers](IMG_8811.png)


The telecommunications (telecom) sector in India is rapidly changing, with more and more telecom businesses being created and many customers deciding to switch between providers. "Churn" refers to the process where customers or subscribers stop using a company's services or products. Understanding the factors that influence keeping a customer as a client in predicting churn is crucial for telecom companies to enhance their service quality and customer satisfaction. As the data scientist on this project, you aim to explore the intricate dynamics of customer behavior and demographics in the Indian telecom sector in predicting customer churn, utilizing two comprehensive datasets from four major telecom partners: Airtel, Reliance Jio, Vodafone, and BSNL:

- `telecom_demographics.csv` contains information related to Indian customer demographics:

| Variable             | Description                                      |
|----------------------|--------------------------------------------------|
| `customer_id `         | Unique identifier for each customer.             |
| `telecom_partner `     | The telecom partner associated with the customer.|
| `gender `              | The gender of the customer.                      |
| `age `                 | The age of the customer.                         |
| `state`                | The Indian state in which the customer is located.|
| `city`                 | The city in which the customer is located.       |
| `pincode`              | The pincode of the customer's location.          |
| `registration_event` | When the customer registered with the telecom partner.|
| `num_dependents`      | The number of dependents (e.g., children) the customer has.|
| `estimated_salary`     | The customer's estimated salary.                 |

- `telecom_usage` contains information about the usage patterns of Indian customers:

| Variable   | Description                                                  |
|------------|--------------------------------------------------------------|
| `customer_id` | Unique identifier for each customer.                         |
| `calls_made` | The number of calls made by the customer.                    |
| `sms_sent`   | The number of SMS messages sent by the customer.             |
| `data_used`  | The amount of data used by the customer.                     |
| `churn`    | Binary variable indicating whether the customer has churned or not (1 = churned, 0 = not churned).|


In [39]:
# Import libraries and methods/functions
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

# Start your code here!

# Load the two CSV files into separate DataFrames. Merge them into a DataFrame named churn_df
demo_df = pd.read_csv('telecom_demographics.csv')

usage_df = pd.read_csv('telecom_usage.csv')

churn_df = demo_df.merge(usage_df, on = 'customer_id')

churn_df.info()
churn_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6500 entries, 0 to 6499
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         6500 non-null   int64 
 1   telecom_partner     6500 non-null   object
 2   gender              6500 non-null   object
 3   age                 6500 non-null   int64 
 4   state               6500 non-null   object
 5   city                6500 non-null   object
 6   pincode             6500 non-null   int64 
 7   registration_event  6500 non-null   object
 8   num_dependents      6500 non-null   int64 
 9   estimated_salary    6500 non-null   int64 
 10  calls_made          6500 non-null   int64 
 11  sms_sent            6500 non-null   int64 
 12  data_used           6500 non-null   int64 
 13  churn               6500 non-null   int64 
dtypes: int64(9), object(5)
memory usage: 761.7+ KB


,customer_id,telecom_partner,gender,age,state,city,pincode,registration_event,num_dependents,estimated_salary,calls_made,sms_sent,data_used,churn
0,15169,Airtel,F,26,Himachal Pradesh,Delhi,667173,2020-03-16,4,85979,75,21,4532,1
1,149207,Airtel,F,74,Uttarakhand,Hyderabad,313997,2022-01-16,0,69445,35,38,723,1
2,148119,Airtel,F,54,Jharkhand,Chennai,549925,2022-01-11,2,75949,70,47,4688,1
3,187288,Reliance Jio,M,29,Bihar,Hyderabad,230636,2022-07-26,3,34272,95,32,10241,1
4,14016,Vodafone,M,45,Nagaland,Bangalore,188036,2020-03-11,4,34157,66,23,5246,1


In [40]:
# Calculate and print churn rate
churn_count = churn_df.loc[churn_df['churn']==1, 'churn'].count()

churn_rate = churn_count/len(churn_df)

print(churn_rate)

0.20046153846153847


In [41]:
# Identify the categorical variables in churn_df
X_cat_columns = churn_df.drop(['customer_id', 'churn'], axis=1).select_dtypes(include=['object']).columns.tolist()

# Convert categorical features in churn_df into features_scaled
enc = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_cat_encoded = enc.fit_transform(churn_df[X_cat_columns])
X_cat_encoded_df = pd.DataFrame(X_cat_encoded, columns=enc.get_feature_names_out(X_cat_columns))

# Select remaining numerical variables in churn_df
X_num_columns = churn_df.drop(['customer_id', 'churn'], axis=1).select_dtypes(include=['int64']).columns.tolist()
X_num_df = churn_df[X_num_columns]

# Merge categorical and numerical variables in churn_df before scaling
features_df = X_num_df.merge(X_cat_encoded_df, left_index=True, right_index=True)

print(features_df)

      age  ...  registration_event_2023-05-03
0      26  ...                            0.0
1      74  ...                            0.0
2      54  ...                            0.0
3      29  ...                            0.0
4      45  ...                            0.0
...   ...  ...                            ...
6495   54  ...                            0.0
6496   69  ...                            0.0
6497   19  ...                            0.0
6498   26  ...                            0.0
6499   19  ...                            0.0

[6500 rows x 1263 columns]


In [42]:
# Perform feature scaling separating the appropriate features and scale them
scr = StandardScaler()

# Define your scaled features and target variable for the churn prediction model
features_scaled = scr.fit_transform(features_df)

target = churn_df['churn']

print(features_scaled)

[[-1.22296979  0.45493603  1.43653887 ... -0.04117252 -0.03283419
  -0.02774568]
 [ 1.6963038  -0.90419473 -1.41134604 ... -0.04117252 -0.03283419
  -0.02774568]
 [ 0.47993981  0.00372937  0.01259641 ... -0.04117252 -0.03283419
  -0.02774568]
 ...
 [-1.64869719 -1.5268359  -1.41134604 ... -0.04117252 -0.03283419
  -0.02774568]
 [-1.22296979  0.89731466  0.72456764 ... -0.04117252 -0.03283419
  -0.02774568]
 [-1.64869719  1.42210486  0.72456764 ... -0.04117252 -0.03283419
  -0.02774568]]


In [43]:
# Split the processed data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

seed = 42

# Train Logistic Regression and Random Forest Classifier models, setting a random seed of 42. Store model predictions in logreg_pred and rf_pred
logreg = LogisticRegression(random_state=seed)
logreg.fit(X_train, y_train)
logreg_pred = logreg.predict(X_test)
print(confusion_matrix(y_test, logreg_pred))
print(classification_report(y_test, logreg_pred))

logreg_score = accuracy_score(y_test, logreg_pred)

rf = RandomForestClassifier(random_state=seed)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

print(confusion_matrix(y_test, rf_pred))
print(classification_report(y_test, rf_pred))

rf_score = accuracy_score(y_test, rf_pred)

# Assign the model's name with higher accuracy ("LogisticRegression" or "RandomForest") to higher_accuracy
higher_accuracy = np.where(logreg_score > rf_score, 'LogisticRegression', 'RandomForest')

[[920 107]
 [245  28]]
              precision    recall  f1-score   support

           0       0.79      0.90      0.84      1027
           1       0.21      0.10      0.14       273

    accuracy                           0.73      1300
   macro avg       0.50      0.50      0.49      1300
weighted avg       0.67      0.73      0.69      1300

[[1026    1]
 [ 273    0]]
              precision    recall  f1-score   support

           0       0.79      1.00      0.88      1027
           1       0.00      0.00      0.00       273

    accuracy                           0.79      1300
   macro avg       0.39      0.50      0.44      1300
weighted avg       0.62      0.79      0.70      1300

